In [1]:
import os

In [2]:
def check_create_dir(path):
    #exists = os.path.isfile(path)
    #exists = os.path.isdir(path)
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

In [4]:
def remove_low_covered(output_dir, sample_list):
    output_dir = os.path.abspath(output_dir)
    group = output_dir.split("/")[-1]
    uncovered_dir = os.path.join(output_dir, "Uncovered")
    check_create_dir(uncovered_dir)
    sample_list_file = os.path.join(output_dir, "sample_list.txt")
    for root, _, files in os.walk(output_dir):
        #Remove recall gvcf to avoid using them to recalibrate
        if root.endswith('GVCF_recal'):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low):
                        os.remove(filename)
        #Place low covered samples in a specific folder to analize them with different parameters
        if root.endswith(group):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low) and name.endswith("fastq.gz"):
                        dest_uncovered_path = os.path.join(uncovered_dir, name)
                        os.rename(filename, dest_uncovered_path)
    if os.path.isfile(sample_list_file):
        edit_sample_list(sample_list_file, sample_list)


In [5]:
#os.rename("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

In [6]:
remove_low_covered("/home/laura/ANALYSIS/SNPTB/190920_HRYC", ["HRYC-11986372","10082989-0-COL11"])

NameError: name 'edit_sample_list' is not defined

In [7]:
sample_list = "/home/laura/ANALYSIS/SNPTB/190920_HRYC/sample_list_test.txt"

In [8]:
def edit_sample_list(file_list, sample_list):
    with open(file_list, 'r') as f:
        content = f.read()
        content_list = content.split('\n')
        while '' in content_list : content_list.remove('')
        
    with open (file_list, 'w+') as fout:
            for line in content_list:
                if line not in sample_list:
                    fout.write(line + "\n")
    

In [9]:
edit_sample_list(sample_list, ["HRYC-11986372","10082989-0-COL11", "10082989-0-COL14"])

FileNotFoundError: [Errno 2] No such file or directory: '/home/laura/ANALYSIS/SNPTB/190920_HRYC/sample_list_test.txt'

In [10]:
folder = "/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC"
samples = ["HIS-29405031","HIS-29405031", "ARG-28441"]

In [13]:
def check_reanalysis(output_dir):
    output_dir = os.path.abspath(output_dir)
    #group = output_dir.split("/")[-1]
    
    bam_dir = os.path.join(output_dir, "Bam")
    vcf_dir = os.path.join(output_dir, "VCF")
    gvcf_dir = os.path.join(output_dir, "GVCF")
    gvcfr_dir = os.path.join(output_dir, "GVCF_recal")
    vcfr_dir = os.path.join(output_dir, "VCF_recal")
    cov_dir = os.path.join(output_dir, "Coverage")
    
    previous_files = [bam_dir, vcf_dir, gvcf_dir, gvcfr_dir]
    
    #check how many folders exist
    file_exist = sum([os.path.exists(x) for x in previous_files]) #True = 1, False = 0
    
    #Handle reanalysis: First time; reanalysis o reanalysis with aditional samples
    if file_exist > 0: #Already analysed
        
        samples_analyzed = os.listdir(bam_dir)
        samples_analyzed = len([ x for x in samples_analyzed if ".bai" not in x and "bqsr" in x])

        samples_fastq = os.listdir(output_dir)
        samples_fastq = len([ x for x in samples_fastq if x.endswith('fastq.gz')]) / 2
        
        if samples_analyzed >= samples_fastq:
            print(MAGENTA + "\nPREVIOUS ANALYSIS DETECTED, NO NEW SEQUENCES ADDED\n" + END_FORMATTING)
        
        else:
            print(MAGENTA + "\nPREVIOUS ANALYSIS DETECTED, NEW SEQUENCES ADDED\n" + END_FORMATTING)
            for root, _, files in os.walk(output_dir):
                    if root ==  gvcf_dir or root == gvcfr_dir or root == vcfr_dir:
                        for name in files:
                            filename = os.path.join(root, name)
                            if (("GVCF_recal" in filename) or ("/VCF_recal" in filename)) and "cohort" in filename and samples_analyzed < 100:
                                os.remove(filename)
                            elif "cohort" in filename and "/GVCF/" in filename:
                                os.remove(filename)
                    elif root == vcf_dir:
                        for name in files:
                            filename = os.path.join(root, name)
                            if "cohort" in filename or filename.endswith(".bed"):
                                os.remove(filename)
                    elif root == cov_dir:
                        for name in files:
                            filename = os.path.join(root, name)
                            if "coverage.tab" in filename or\
                            "poorly_covered.bed" in filename:
                                os.remove(filename)
            #print(file_exist, samples_analyzed, samples_fastq)

In [12]:
check_reanalysis(folder)

NameError: name 'MAGENTA' is not defined

In [ ]:
non_genotyped_count = sum(non_genotyped) 

In [21]:
def remove_low_covered_mixed(output_dir, sample_list, type_remove):
    output_dir = os.path.abspath(output_dir)
    group = output_dir.split("/")[-1]
    uncovered_dir = os.path.join(output_dir, type_remove) #Uncovered or Mixed
    check_create_dir(uncovered_dir)

    sample_list_file = os.path.join(output_dir, "sample_list.txt")
    
    for root, _, files in os.walk(output_dir):
        #Any previous file created except for Table for mixed samples
        # and Species for both uncovered and mixed
        if root.endswith('GVCF_recal') or root.endswith('Coverage') \
        or root.endswith('VCF') or root.endswith('VCF_recal') \
        or root.endswith('Bam') or root.endswith('GVCF') \
        or root.endswith('Table'):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low):
                        print(filename)
 
        #Place low covered samples in a specific folder to analize them with different parameters
        if root.endswith(group):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    sample_lowbar = sample_low + "_"
                    if name.startswith(sample_lowbar) and name.endswith("fastq.gz"):
                        print(sample_lowbar)
                        dest_uncovered_path = os.path.join(uncovered_dir, name)
                        if os.path.isfile(sample_list_file):
                            print(filename, dest_uncovered_path)
    if os.path.isfile(sample_list_file):
        edit_sample_list(sample_list_file, sample_list)


In [22]:
pipeline_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC"
list_s = ['937_CGTACTAG-AGAGTAGA_L001','511-51879','9','145_P_002885','MGIT268','MGIT42','AB-40701','131_P_137862']

In [23]:
remove_low_covered_mixed(pipeline_folder, list_s, "Uncovered")

MGIT42_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/MGIT42_S1_L001_R1_001.fastq.gz /home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Uncovered/MGIT42_S1_L001_R1_001.fastq.gz
AB-40701_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/AB-40701_S16_L001_R2_001.fastq.gz /home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Uncovered/AB-40701_S16_L001_R2_001.fastq.gz
937_CGTACTAG-AGAGTAGA_L001_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/937_CGTACTAG-AGAGTAGA_L001_R2.fastq.gz /home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Uncovered/937_CGTACTAG-AGAGTAGA_L001_R2.fastq.gz
145_P_002885_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/145_P_002885_S49_L002_R1_001.fastq.gz /home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Uncovered/145_P_002885_S49_L002_R1_001.fastq.gz
AB-40701_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/AB-40701_S16_L001_R1_001.fastq.gz /home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Uncovered/AB-40701_S16_L001_R1_001.fastq.gz
9_
/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/9_S4_L001_R2_001.fastq.gz /hom